In [1]:
from skmultilearn.dataset import load_dataset
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
import sklearn.metrics as metrics

In [2]:
# Loading Datasets
X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ = load_dataset('emotions', 'test')

emotions:train - exists, not redownloading
emotions:test - exists, not redownloading


In [3]:
# Binary Revelance: Transforms a multi-label classification problem 
# with L labels into L single-label separate binary classification problems.
# Model used SVC()

clf = BinaryRelevance(
    classifier=SVC(),
    require_dense=[False, True]
)

clf.fit(X_train, y_train)


BinaryRelevance(classifier=SVC(), require_dense=[False, True])

In [4]:
# print the models used for each binery classifier
clf.classifiers_

[SVC(), SVC(), SVC(), SVC(), SVC(), SVC()]

In [5]:
# predict function
prediction = clf.predict(X_test)
prediction

<202x6 sparse matrix of type '<class 'numpy.int64'>'
	with 27 stored elements in Compressed Sparse Column format>

In [6]:
# metric evaluation
print(metrics.hamming_loss(y_test, prediction))
print(metrics.accuracy_score(y_test, prediction))

0.3250825082508251
0.01485148514851485


In [7]:
from skmultilearn.adapt import MLkNN # -> c'è un errore sulla libreria che non permette 
                                     #    il passaggio del secondo parametro e da errore.
                                     #    https://github.com/scikit-multilearn/scikit-multilearn/issues/230
from sklearn.model_selection import GridSearchCV


parameters = {'k': range(1,3), 's': [0.5, 0.7, 1.0]}

clf = BinaryRelevance(
    classifier=SVC(),
    require_dense=[False, True]
)

clf = GridSearchCV(MLkNN(), parameters, scoring='accuracy')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)

ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\adapt\mlknn.py", line 218, in fit
    self._cond_prob_true, self._cond_prob_false = self._compute_cond(X, self._label_cache)
  File "C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\adapt\mlknn.py", line 165, in _compute_cond
    self.knn_ = NearestNeighbors(self.k).fit(X)
TypeError: NearestNeighbors.__init__() takes 1 positional argument but 2 were given


In [8]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC

parameters = [
    {
        'classifier': [BernoulliNB()],
        'classifier__alpha': [0.7, 1.0],
    },
    {
        'classifier': [SVC()],
        'classifier__kernel': ['rbf', 'linear'],
    },
]

clf = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)

{'classifier': SVC(kernel='linear'), 'classifier__kernel': 'linear'} 0.2711457319052256


In [11]:
from skmultilearn.problem_transform import ClassifierChain, LabelPowerset
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.cluster import NetworkXLabelGraphClusterer
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

from sklearn.svm import SVC

# parameters = {
# classifier: which takes a parameter - a classifier for transforming multi-label classification problem to a single-label classification, we will decide between the Label Powerset and Classifier Chains
# classifier__classifier: which is the base classifier for the transformation strategy, we will use random forests here
# classifier__classifier__*: the number of trees to be used in the forest, will be passed to the random forest object
# clusterer: a label space partitioning class, we will decide between two approaches provided by the NetworkX library.
#}


parameters = {
    'classifier': [LabelPowerset(), ClassifierChain()],    
    'classifier__classifier': [RandomForestClassifier()],  
    'classifier__classifier__n_estimators': [10, 20, 50],
    'clusterer' : [
        NetworkXLabelGraphClusterer(LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False), 'louvain'),
        NetworkXLabelGraphClusterer(LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False), 'lpa')
    ]
}

clf = GridSearchCV(LabelSpacePartitioningClassifier(), parameters, scoring = 'f1_macro')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)

C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged ne

C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged ne

{'classifier': LabelPowerset(classifier=RandomForestClassifier(n_estimators=50),
              require_dense=[True, True]), 'classifier__classifier': RandomForestClassifier(n_estimators=50), 'classifier__classifier__n_estimators': 50, 'clusterer': <skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x0000015267E0FFD0>} 0.6057297622741336


In [14]:
# Binary Relevance with Random Forest of Decision Trees -> RFDTBR
classifier = BinaryRelevance(
    classifier = RandomForestClassifier(n_estimators=50),
    require_dense = [False, True]
)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

# metric evaluation
print(metrics.hamming_loss(y_test, prediction))
print(metrics.jaccard_score( average='macro'))

# Ensemble of Binary Relevance with Decision Tree -> EBRDC
classifier = LabelSpacePartitioningClassifier(
    classifier = BinaryRelevance(classifier= DecisionTreeClassifier()),
    clusterer =  NetworkXLabelGraphClusterer(LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False), 'louvain')
)
classifier.fit(X_train,y_train)
predictions = classifier.predict(X_test)

print(metrics.hamming_loss(y_test, prediction))
print(metrics.jaccard_score(y_test, prediction, average='macro'))

0.3250825082508251
0.024922118380062305
0.3250825082508251
0.024922118380062305


In [20]:
parameters = {
    'classifier': [RandomForestClassifier()],  
    'classifier__n_estimators': [10, 20, 50],
}

clf = GridSearchCV(BinaryRelevance(), parameters, scoring = 'jaccard_macro')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)

parameters = {
    'classifier': [BinaryRelevance()],    
    'classifier__classifier': [DecisionTreeClassifier()],  
    'clusterer' : [
        NetworkXLabelGraphClusterer(LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False), 'louvain'),
        NetworkXLabelGraphClusterer(LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False), 'lpa')
    ]
}

clf = GridSearchCV(LabelSpacePartitioningClassifier(), parameters, scoring = 'jaccard_macro')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)

{'classifier': RandomForestClassifier(n_estimators=50), 'classifier__n_estimators': 50} 0.4272443760337591


C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged ne

{'classifier': BinaryRelevance(classifier=DecisionTreeClassifier(), require_dense=[True, True]), 'classifier__classifier': DecisionTreeClassifier(), 'clusterer': <skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000001526A5CEE90>} 0.4055592378643344


C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
C:\ProgramData\anaconda3\lib\site-packages\skmultilearn\cluster\networkx.py:170: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([list(i) for i in asyn_lpa_communities(self.graph_, 'weight')])
